In [77]:
import pandas as pd
import numpy as np
from datetime import date
import unidecode
import json

In [2]:
ls

deputies.json    Leg_2017_Resultats_T1_c.xlsx*    Leg_2017_Resultats_T2_c.xlsx
ETL_json1.ipynb  Leg_2017_Resultats_T2_c.csv      liste.csv*
french.json*     Leg_2017_Resultats_T2_c.numbers


In [3]:
res = pd.read_csv("Leg_2017_Resultats_T2_c.csv", sep=";")
htpr = pd.read_csv("liste.csv")

## Préparation de la table de résultat

In [4]:
res = res.rename(columns={"Né(e)_le":"date"})

In [5]:
def calculate_age(born):
    today = date.today()
    born = born.split("/")
    born_dict = {"year": int("19"+born[2]) ,"day":int(born[1]),"month":int(born[0]) }
    return today.year - born_dict["year"] - ((today.month, today.day) < (born_dict["month"], born_dict["day"]))

In [6]:
calculate_age(res["date"][0])

43

In [7]:
res["age"] = res["date"].map(lambda x : calculate_age(x))

In [8]:
res["nom"] = res["Désignation"].map(lambda x: unidecode.unidecode(x.split(" ")[1].strip()).upper())
res["prenom"] = res["Désignation"].map(lambda x:unidecode.unidecode(x.split(" ")[2].strip()).upper())

In [9]:
res["sexe"] = res["Désignation"].map(lambda x : "M" if x.split(" ")[0]=="M." else "F")

In [10]:
res.head()

,Dpt,Nom_de_la_Circonscription,Désignation,Nuance,Score,Sortant,Perso.,date,Profession,Tour,age,nom,prenom,sexe
0,ZA,1ère circonscription,M. SERVA Olivier,REM,"61,74 %",NaN,NaN,6/21/74,34 - Expert comptable,2,43,SERVA,OLIVIER,M
1,ZA,2ème circonscription,Mme BENIN Justine,DVG,"64,26 %",NaN,NaN,3/12/75,48 - Fonctionnaire de catégorie B,2,42,BENIN,JUSTINE,F
2,ZA,3ème circonscription,M. MATHIASIN Max,DVG,"65,15 %",NaN,NaN,2/24/56,52 - Employé (autres entrep. publiques),2,61,MATHIASIN,MAX,M
3,ZA,4ème circonscription,Mme VAINQUEUR-CHRISTOPHE Hélène,SOC,"61,61 %",NaN,NaN,5/6/56,28 - Pharmacien,2,61,VAINQUEUR-CHRISTOPHE,HELENE,F
4,ZB,1ère circonscription,Mme MANIN Josette,DVG,"54,89 %",NaN,NaN,3/16/50,61 - Retraité salarié privé,2,67,MANIN,JOSETTE,F


## Préparation de la table de la haute autorité

In [11]:
htpr["nom"] = htpr["nom"].map(lambda x:unidecode.unidecode(x.strip()).upper())
htpr["prenom"] = htpr["prenom"].map(lambda x:unidecode.unidecode(x.strip()).upper())

In [12]:
htpr.head()

,civilite,prenom,nom,classement,type_mandat,qualite,type_document,departement,date_publication,nom_fichier,url_dossier,id_origine,url_photo
0,Mme,ABBASSIA,HAKEM,hakemaaaabbassia4615,commune,Adjointe au maire de Nantes,di,44,2016-07-07,hakem-abbassia-di-commune-nantes.pdf,/pages_nominatives/hakem-abbassia,NaN,NaN
1,M.,ABDEL,GHEZALI,ghezaliaaaabdel3656,commune,Adjoint au maire de Besançon,di,25,2016-07-07,ghezali-abdel-di-commune-besancon.pdf,/pages_nominatives/ghezali-abdel,NaN,NaN
2,M.,ABDEL-MADJID,SADI,sadiaaaabdelmadjid8238,departement,Vice-Président du conseil départemental de la ...,di,93,2016-10-28,sadi-abdel-madjid-di-departement-93.pdf,/pages_nominatives/sadi-abdel-madjid-8238,NaN,NaN
3,M.,ABDOURAHAMANE,SOILIHI,soilihiaaaabdourahamane8213,senateur,Sénateur de Mayotte,dia,976,2014-07-23,soilihi-abdourahamane-dia-senateur-976.pdf,/pages_nominatives/soilihi-abdourahamane,11071M,http://www.senat.fr/senimg/soilihi_abdourahama...
4,M.,ABDOURAHAMANE,SOILIHI,soilihiaaaabdourahamane8213,senateur,Sénateur de Mayotte,dsp,976,2015-07-10,soilihi-abdourahamane-dsp-senateur-976.pdf,/pages_nominatives/soilihi-abdourahamane,11071M,http://www.senat.fr/senimg/soilihi_abdourahama...


In [13]:
htpr[htpr["nom"]=="BAYROU"]

,civilite,prenom,nom,classement,type_mandat,qualite,type_document,departement,date_publication,nom_fichier,url_dossier,id_origine,url_photo
1770,M.,FRANCOIS,BAYROU,bayrouaaafrancois58,commune,Maire de Pau,di,64,2015-12-17,bayrou-francois-di-commune-pau.pdf,/pages_nominatives/bayrou-francois,NaN,NaN


In [14]:
# Garde que les personnes qui ont un dossier renseignée 
htpr = htpr[~htpr["url_dossier"].isnull()]
htpr["link"] = htpr["nom_fichier"].map(lambda x : "http://www.hatvp.fr/livraison/dossiers/"+str(x))

In [15]:
print(htpr.shape)

(5563, 14)


In [16]:
# Prendre la photo la plus récente 
sub_htrp = htpr.dropna()
sub_htrp = sub_htrp.sort_values(["nom","prenom","date_publication"], ascending=False)
sub_htrp = sub_htrp.drop_duplicates(["nom","prenom"])
sub_htrp = sub_htrp.rename(columns={"url_photo": "link_pict"})

In [17]:
htpr = pd.merge(htpr, sub_htrp[["nom","prenom","link_pict"]],how="left", on=["nom","prenom"])

In [18]:
htpr.head()

,civilite,prenom,nom,classement,type_mandat,qualite,type_document,departement,date_publication,nom_fichier,url_dossier,id_origine,url_photo,link,link_pict
0,Mme,ABBASSIA,HAKEM,hakemaaaabbassia4615,commune,Adjointe au maire de Nantes,di,44,2016-07-07,hakem-abbassia-di-commune-nantes.pdf,/pages_nominatives/hakem-abbassia,NaN,NaN,http://www.hatvp.fr/livraison/dossiers/hakem-a...,NaN
1,M.,ABDEL,GHEZALI,ghezaliaaaabdel3656,commune,Adjoint au maire de Besançon,di,25,2016-07-07,ghezali-abdel-di-commune-besancon.pdf,/pages_nominatives/ghezali-abdel,NaN,NaN,http://www.hatvp.fr/livraison/dossiers/ghezali...,NaN
2,M.,ABDEL-MADJID,SADI,sadiaaaabdelmadjid8238,departement,Vice-Président du conseil départemental de la ...,di,93,2016-10-28,sadi-abdel-madjid-di-departement-93.pdf,/pages_nominatives/sadi-abdel-madjid-8238,NaN,NaN,http://www.hatvp.fr/livraison/dossiers/sadi-ab...,NaN
3,M.,ABDOURAHAMANE,SOILIHI,soilihiaaaabdourahamane8213,senateur,Sénateur de Mayotte,dia,976,2014-07-23,soilihi-abdourahamane-dia-senateur-976.pdf,/pages_nominatives/soilihi-abdourahamane,11071M,http://www.senat.fr/senimg/soilihi_abdourahama...,http://www.hatvp.fr/livraison/dossiers/soilihi...,http://www.senat.fr/senimg/soilihi_abdourahama...
4,M.,ABDOURAHAMANE,SOILIHI,soilihiaaaabdourahamane8213,senateur,Sénateur de Mayotte,dsp,976,2015-07-10,soilihi-abdourahamane-dsp-senateur-976.pdf,/pages_nominatives/soilihi-abdourahamane,11071M,http://www.senat.fr/senimg/soilihi_abdourahama...,http://www.hatvp.fr/livraison/dossiers/soilihi...,http://www.senat.fr/senimg/soilihi_abdourahama...


In [19]:
print(htpr.shape)

(5563, 15)


In [20]:
# garder la déclaration la plus récente 
htpr = htpr.sort_values(["nom","prenom","date_publication"], ascending=False)

In [21]:
htpr.head()

,civilite,prenom,nom,classement,type_mandat,qualite,type_document,departement,date_publication,nom_fichier,url_dossier,id_origine,url_photo,link,link_pict
589,M.,BERNARD,ZUNINO,zuninoaaabernard9220,commune,Maire de Saint-Michel-sur-Orge,di,91,2015-12-17,zunino-bernard-di-commune-saint-michel-sur-org...,/pages_nominatives/zunino-bernard,NaN,NaN,http://www.hatvp.fr/livraison/dossiers/zunino-...,NaN
4090,M.,MICHEL,ZUMKELLER,zumkelleraaamichel9238,depute,Député du Territoire de Belfort,dsp,90,2015-07-10,zumkeller-michel-dsp-depute-90.pdf,/pages_nominatives/zumkeller-michel,267330,http://www2.assemblee-nationale.fr/static/trib...,http://www.hatvp.fr/livraison/dossiers/zumkell...,http://www2.assemblee-nationale.fr/static/trib...
4089,M.,MICHEL,ZUMKELLER,zumkelleraaamichel9238,depute,Député du Territoire de Belfort,dia,90,2014-07-23,zumkeller-michel-dia-depute-90.pdf,/pages_nominatives/zumkeller-michel,267330,http://www2.assemblee-nationale.fr/static/trib...,http://www.hatvp.fr/livraison/dossiers/zumkell...,http://www2.assemblee-nationale.fr/static/trib...
3764,Mme,MARIE,ZUCCARELLI,ZUCCARELLI Marie10371,departement,Conseillère départementale de la Corse-du-Sud,di,2A,NaN,NaN,/pages_nominatives/zuccarelli-marie,NaN,NaN,http://www.hatvp.fr/livraison/dossiers/nan,NaN
2933,M.,JEAN-MARIE,ZOELLE,zoelleaaajeanmarie9213,commune,Maire de Saint-Louis,di,57,2015-12-17,zoelle-jean-marie-di-commune-saint-louis.pdf,/pages_nominatives/zoelle-jean-marie,NaN,NaN,http://www.hatvp.fr/livraison/dossiers/zoelle-...,NaN


In [22]:
htpr = htpr.drop_duplicates(["nom","prenom"], keep='first')

In [23]:
htpr.shape

(3417, 15)

## Préparation table résultat premier tour 

In [24]:
vrai_res = pd.read_excel("Leg_2017_Resultats_T1_c.xlsx", sheetname="Circo. leg. T1", skiprows=2)

In [25]:
vrai_res["code"] = vrai_res.apply(lambda row : str(row["Code du département"])+'|'
                                  +str(row["Code de la circonscription"]), axis=1)

vrai_res = vrai_res.set_index("code")

one_df = vrai_res[["Nom","Nom.1","Nom.2","Nom.3","Nom.4","Nom.5","Nom.6",
         "Nom.7","Nom.8","Nom.9","Nom.10","Nom.11","Nom.12","Nom.13","Nom.14","Nom.15","Nom.16","Nom.17","Nom.18",
         "Nom.19","Nom.20","Nom.21","Nom.22","Nom.23","Nom.24",
         "Nom.25","Nom.26"]].stack()
one_df = one_df.rename("nom")
one_df = one_df.reset_index()
del one_df["level_1"]

two_df = vrai_res[["Prénom","Prénom.1","Prénom.2","Prénom.3","Prénom.4","Prénom.5","Prénom.6",
         "Prénom.7","Prénom.8","Prénom.9","Prénom.10","Prénom.11","Prénom.12","Prénom.13","Prénom.14",
        "Prénom.15","Prénom.16","Prénom.17","Prénom.18",
         "Prénom.19","Prénom.20","Prénom.21","Prénom.22","Prénom.23","Prénom.24",
         "Prénom.25","Prénom.26"]].stack()
two_df = two_df.rename("prenom")
two_df = two_df.reset_index()
del two_df["code"]
del two_df["level_1"]

tree_df = vrai_res[["% Voix/Exp","% Voix/Exp.1","% Voix/Exp.2","% Voix/Exp.3","% Voix/Exp.4","% Voix/Exp.5",
                    "% Voix/Exp.6",
         "% Voix/Exp.7","% Voix/Exp.8","% Voix/Exp.9","% Voix/Exp.10","% Voix/Exp.11","% Voix/Exp.12",
                    "% Voix/Exp.13","% Voix/Exp.14",
        "% Voix/Exp.15","% Voix/Exp.16","% Voix/Exp.17","% Voix/Exp.18",
         "% Voix/Exp.19","% Voix/Exp.20","% Voix/Exp.21","% Voix/Exp.22","% Voix/Exp.23","% Voix/Exp.24",
         "% Voix/Exp.25","% Voix/Exp.26"]].stack()
tree_df = tree_df.rename("score_exp")
tree_df = tree_df.reset_index()
del tree_df["code"]
del tree_df["level_1"]


quar_df = vrai_res[["% Voix/Ins","% Voix/Ins.1","% Voix/Ins.2","% Voix/Ins.3","% Voix/Ins.4","% Voix/Ins.5",
                    "% Voix/Ins.6",
         "% Voix/Ins.7","% Voix/Ins.8","% Voix/Ins.9","% Voix/Ins.10","% Voix/Ins.11","% Voix/Ins.12",
                    "% Voix/Ins.13","% Voix/Ins.14",
        "% Voix/Ins.15","% Voix/Ins.16","% Voix/Ins.17","% Voix/Ins.18",
         "% Voix/Ins.19","% Voix/Ins.20","% Voix/Ins.21","% Voix/Ins.22","% Voix/Ins.23","% Voix/Ins.24",
         "% Voix/Ins.25","% Voix/Ins.26"]].stack()
quar_df = quar_df.rename("score_ins")
quar_df = quar_df.reset_index()
del quar_df["code"]
del quar_df["level_1"]

df_resultat = pd.concat([one_df, two_df, tree_df, quar_df], axis=1)

In [26]:
df_resultat["nom"] = df_resultat["nom"].map(lambda x: unidecode.unidecode(x.strip()).upper())
df_resultat["prenom"] = df_resultat["prenom"].map(lambda x: unidecode.unidecode(x.strip()).upper())

In [27]:
df_resultat.head()

,code,nom,prenom,score_exp,score_ins
0,1|1,LEPAGNOT,MAUDE,0.73,0.35
1,1|1,FONTAINE,JACQUES,1.64,0.79
2,1|1,MARTIN ZEMLIK,FABRINE,9.70,4.68
3,1|1,BLATRIX-CONTAT,FLORENCE,9.23,4.46
4,1|1,RAIMONDO,LAURANE,1.41,0.68


## Merge table 

In [28]:
print(res.shape)

(577, 14)


In [29]:
json = pd.merge(res, htpr[["nom","prenom","link","link_pict"]], how="left", on=["nom","prenom"])

In [30]:
json.shape

(577, 16)

In [31]:
# 40 NOMS qui déconne à régler
json.ix[json["Désignation"]=="M. AZEROT Bruno Nestor","prenom"]="BRUNO NESTOR"
json.ix[json["Désignation"]=="M. RATENON Jean Hugues","prenom"]="JEAN HUGUES"
json.ix[json["Désignation"]=="M. BROTHERSON Moetai, Charles","prenom"]="MOETAI, CHARLES"

json.ix[json["Désignation"]=="M. EL GUERRAB M'Jid","nom"]="EL GUERRAB"
json.ix[json["Désignation"]=="M. EL GUERRAB M'Jid","prenom"]="M'JID"

json.ix[json["Désignation"]=="Mme LAKRAFI Amal Amélia","prenom"]="AMAL AMELIA"

json.ix[json["Désignation"]=="M. DE LA VERPILLIÈRE Charles","nom"]="DE LA VERPILLIERE"
json.ix[json["Désignation"]=="M. DE LA VERPILLIÈRE Charles","prenom"]="CHARLES"

json.ix[json["Désignation"]=="Mme VALETTA ARDISSON Alexandra","nom"]="VALETTA ARDISSON"
json.ix[json["Désignation"]=="Mme VALETTA ARDISSON Alexandra","prenom"]="ALEXANDRA"

json.ix[json["Désignation"]=="M. LE VIGOUREUX Fabrice","nom"]="LE VIGOUREUX"
json.ix[json["Désignation"]=="M. LE VIGOUREUX Fabrice","prenom"]="FABRICE"

json.ix[json["Désignation"]=="Mme DE COURSON Yolaine","nom"]="DE COURSON"
json.ix[json["Désignation"]=="Mme DE COURSON Yolaine","prenom"]="YOLAINE"

json.ix[json["Désignation"]=="M. LE FUR Marc","nom"]="LE FUR"
json.ix[json["Désignation"]=="M. LE FUR Marc","prenom"]="MARC"

json.ix[json["Désignation"]=="Mme DE LAVERGNE Célia","nom"]="DE LAVERGNE"
json.ix[json["Désignation"]=="Mme DE LAVERGNE Célia","prenom"]="CELIA"

json.ix[json["Désignation"]=="M. LE MAIRE Bruno","nom"]="LE MAIRE"
json.ix[json["Désignation"]=="M. LE MAIRE Bruno","prenom"]="BRUNO"

json.ix[json["Désignation"]=="Mme DE LA RAUDIERE Laure","nom"]="DE LA RAUDIERE"
json.ix[json["Désignation"]=="Mme DE LA RAUDIERE Laure","prenom"]="LAURE"

json.ix[json["Désignation"]=="Mme LE MEUR Annaïg","nom"]="LE MEUR"
json.ix[json["Désignation"]=="Mme LE MEUR Annaïg","prenom"]="ANNAIG"

json.ix[json["Désignation"]=="M. LE GAC Didier","nom"]="LE GAC"
json.ix[json["Désignation"]=="M. LE GAC Didier","prenom"]="DIDIER"

json.ix[json["Désignation"]=="Mme LE FEUR Sandrine","nom"]="LE FEUR"
json.ix[json["Désignation"]=="Mme LE FEUR Sandrine","prenom"]="SANDRINE"

json.ix[json["Désignation"]=="M. LE BOHEC Gaël","nom"]="LE BOHEC"
json.ix[json["Désignation"]=="M. LE BOHEC Gaël","prenom"]="GAEL"

json.ix[json["Désignation"]=="M. DE RUGY François","nom"]="DE RUGY"
json.ix[json["Désignation"]=="M. DE RUGY François","prenom"]="FRANCOIS"
json.ix[json["Désignation"]=="Mme EL HAÏRY Sarah","nom"]="EL HAIRY"
json.ix[json["Désignation"]=="Mme EL HAÏRY Sarah","prenom"]="SARAH"
json.ix[json["Désignation"]=="Mme DUFEU SCHUBERT Audrey","nom"]="DUFEU SCHUBERT"
json.ix[json["Désignation"]=="Mme DUFEU SCHUBERT Audrey","prenom"]="AUDREY"

json.ix[json["Désignation"]=="M. DE GANAY Claude","nom"]="DE GANAY"
json.ix[json["Désignation"]=="M. DE GANAY Claude","prenom"]="CLAUDE"
json.ix[json["Désignation"]=="M. MOREL A L'HUISSIER Pierre","nom"]="MOREL A L'HUISSIER"
json.ix[json["Désignation"]=="M. MOREL A L'HUISSIER Pierre","prenom"]="PIERRE"
json.ix[json["Désignation"]=="M. DE COURSON Charles","nom"]="DE COURSON"
json.ix[json["Désignation"]=="M. DE COURSON Charles","prenom"]="CHARLES"

json.ix[json["Désignation"]=="Mme LE PEIH Nicole","nom"]="LE PEIH"
json.ix[json["Désignation"]=="Mme LE PEIH Nicole","prenom"]="NICOLE"
json.ix[json["Désignation"]=="M. DI FILIPPO Fabien","nom"]="DI FILIPPO"
json.ix[json["Désignation"]=="M. DI FILIPPO Fabien","prenom"]="FABIEN"
json.ix[json["Désignation"]=="M. DI POMPEO Christophe","nom"]="DI POMPEO"
json.ix[json["Désignation"]=="M. DI POMPEO Christophe","prenom"]="CHRISTOPHE"

json.ix[json["Désignation"]=="Mme CATTELOT Anne Laure","prenom"]="ANNE LAURE"
json.ix[json["Désignation"]=="Mme DE TEMMERMAN Jennifer","nom"]="DE TEMMERMAN"
json.ix[json["Désignation"]=="Mme DE TEMMERMAN Jennifer","prenom"]="JENNIFER"
json.ix[json["Désignation"]=="Mme LE PEN Marine","nom"]="LE PEN"
json.ix[json["Désignation"]=="Mme LE PEN Marine","prenom"]="MARINE"

json.ix[json["Désignation"]=="Mme PIRÈS BEAUNE Christine","nom"]="PIRES BEAUNE"
json.ix[json["Désignation"]=="Mme PIRÈS BEAUNE Christine","prenom"]="CHRISTINE"
json.ix[json["Désignation"]=="Mme BESSOT BALLOT Barbara","nom"]="BESSOT BALLOT"
json.ix[json["Désignation"]=="Mme BESSOT BALLOT Barbara","prenom"]="BARBARA"

json.ix[json["Désignation"]=="M. LE FOLL Stéphane","nom"]="LE FOLL"
json.ix[json["Désignation"]=="M. LE FOLL Stéphane","prenom"]="STEPHANE"
json.ix[json["Désignation"]=="M. LE GENDRE Gilles","nom"]="LE GENDRE"
json.ix[json["Désignation"]=="M. LE GENDRE Gilles","prenom"]="GILLES"

json.ix[json["Désignation"]=="Mme DE SARNEZ Marielle","nom"]="DE SARNEZ"
json.ix[json["Désignation"]=="Mme DE SARNEZ Marielle","prenom"]="MARIELLE"
json.ix[json["Désignation"]=="Mme FIRMIN LE BODO Agnès","nom"]="FIRMIN LE BODO"
json.ix[json["Désignation"]=="Mme FIRMIN LE BODO Agnès","prenom"]="AGNES"

json.ix[json["Désignation"]=="Mme DE VAUCOULEURS Michèle","nom"]="DE VAUCOULEURS"
json.ix[json["Désignation"]=="Mme DE VAUCOULEURS Michèle","prenom"]="MICHELE"
json.ix[json["Désignation"]=="Mme ROMEIRO DIAS Laëtitia","nom"]="ROMEIRO DIAS"
json.ix[json["Désignation"]=="Mme ROMEIRO DIAS Laëtitia","prenom"]="LAETITIA"

json.ix[json["Désignation"]=="Mme DE MONTCHALIN Amélie","nom"]="DE MONTCHALIN"
json.ix[json["Désignation"]=="Mme DE MONTCHALIN Amélie","prenom"]="AMELIE"
json.ix[json["Désignation"]=="Mme LE GRIP Constance","nom"]="LE GRIP"
json.ix[json["Désignation"]=="Mme LE GRIP Constance","prenom"]="CONSTANCE"

json.ix[json["Désignation"]=="M. MBAYE Jean François","nom"]="MBAYE"
json.ix[json["Désignation"]=="M. MBAYE Jean François","prenom"]="JEAN FRANCOIS"
json.ix[json["Désignation"]=="M. DA SILVA Dominique","nom"]="DA SILVA"
json.ix[json["Désignation"]=="M. DA SILVA Dominique","prenom"]="DOMINIQUE"

In [32]:
json_t = pd.merge(json, df_resultat, on=["nom","prenom"])

In [33]:
json_t.shape

(577, 19)

In [34]:
json_t["smart_prof"] = json_t["Profession"].map(lambda x :x.split("-")[1].strip())

In [35]:
json_t["corr_dept"] = json_t["Dpt"]
json_t.ix[json_t["corr_dept"]=="ZA", "corr_dept"]="971"
json_t.ix[json_t["corr_dept"]=="ZB", "corr_dept"]="972"
json_t.ix[json_t["corr_dept"]=="ZC", "corr_dept"]="973"
json_t.ix[json_t["corr_dept"]=="ZD", "corr_dept"]="974"
json_t.ix[json_t["corr_dept"]=="ZM", "corr_dept"]="985"
json_t.ix[json_t["corr_dept"]=="ZN", "corr_dept"]="988"
json_t.ix[json_t["corr_dept"]=="ZP", "corr_dept"]="987"
json_t.ix[json_t["corr_dept"]=="ZS", "corr_dept"]="975"
json_t.ix[json_t["corr_dept"]=="ZW", "corr_dept"]="986"
json_t.ix[json_t["corr_dept"]=="ZX", "corr_dept"]="979"
json_t.ix[json_t["corr_dept"]=="ZZ", "corr_dept"]="99"

In [36]:
json_t["new_code"] = json_t.apply(lambda row : str(row["corr_dept"])+'|'
                                  +str(row["code"].split("|")[1]), axis=1)

In [40]:
json_t.shape

(577, 22)

In [38]:
json_2 = pd.read_excel("/home/brehelin/Documents/Elections/legislatives/data/comparaison_prediction_reel.xlsx", skiprows=2)

In [39]:
json_2.head()

,Dpt,N_circo,Code,Unnamed: 3,Unnamed: 4,Désignation,Nuance,Score,Sortant,Perso.,...,Accès T2 ?,Elu ?,circo_leg_meme_nuance,circo_pres_meme_nuance,score_bloc_pres,depute_sortant,ancien_depute,au_gouvernement,ancien_ministre,membre_majorite
0,971,1,971|1,SERVA,Olivier,M. SERVA Olivier,REM,0.6174,NaN,NaN,...,O,O,0.0,1.0,0.179,0.0,0.0,0.0,0.0,1.0
1,971,2,971|2,BENIN,Justine,Mme BENIN Justine,DVG,0.6426,NaN,NaN,...,N,N,1.0,0.0,0.049,0.0,0.0,0.0,0.0,0.0
2,971,3,971|3,MATHIASIN,Max,M. MATHIASIN Max,DVG,0.6515,NaN,NaN,...,O,O,1.0,0.0,0.059,0.0,0.0,0.0,0.0,0.0
3,971,4,971|4,VAINQUEUR-CHRISTOPHE,Hélène,Mme VAINQUEUR-CHRISTOPHE Hélène,SOC,0.6161,NaN,NaN,...,O,O,1.0,0.0,0.070,0.0,0.0,0.0,0.0,0.0
4,972,1,972|1,MANIN,Josette,Mme MANIN Josette,DVG,0.5489,NaN,NaN,...,N,N,0.0,0.0,0.054,0.0,0.0,0.0,0.0,0.0


In [52]:
json_2 = json_2.rename(columns={"Code":"new_code",
                              '% Tour 1':"pred_tour_1","Accès T2 ?":"pred_acces_2",
                               "Elu ?":"pred_elu"})

In [158]:
json_f = pd.merge(json_t, json_2[["new_code", 'pred_tour_1', 'pred_acces_2', 'pred_elu',
        'circo_leg_meme_nuance',
       'circo_pres_meme_nuance', 'score_bloc_pres', 'depute_sortant',
       'ancien_depute', 'au_gouvernement', 'ancien_ministre',
       'membre_majorite',]], on=["new_code"])

In [159]:
data_circo = pd.read_excel("/home/brehelin/Documents/Elections/legislatives/data/dataframe_elections.xlsx")

In [160]:
data_circo = data_circo[["code","revenus_med","inscrits","part_impose","chom_tot","etrangers"]]

In [161]:
data_circo = data_circo.drop_duplicates(["code"], keep='first')
data_circo = data_circo.rename(columns={"code":"new_code"})

In [162]:
json_f = pd.merge(json_f, data_circo, on=["new_code"])

In [163]:
json_f.shape

(577, 38)

In [164]:
json_f.columns

Index(['Dpt', 'Nom_de_la_Circonscription', 'Désignation', 'Nuance', 'Score',
       'Sortant', 'Perso.', 'date', 'Profession', 'Tour', 'age', 'nom',
       'prenom', 'sexe', 'link', 'link_pict', 'code', 'score_exp', 'score_ins',
       'smart_prof', 'corr_dept', 'new_code', 'pred_tour_1', 'pred_acces_2',
       'pred_elu', 'circo_leg_meme_nuance', 'circo_pres_meme_nuance',
       'score_bloc_pres', 'depute_sortant', 'ancien_depute', 'au_gouvernement',
       'ancien_ministre', 'membre_majorite', 'revenus_med', 'inscrits',
       'part_impose', 'chom_tot', 'etrangers'],
      dtype='object')

In [92]:
#json_f = json_f.set_index(["new_code",'Désignation'])

In [165]:
json_f

,Dpt,Nom_de_la_Circonscription,Désignation,Nuance,Score,Sortant,Perso.,date,Profession,Tour,...,depute_sortant,ancien_depute,au_gouvernement,ancien_ministre,membre_majorite,revenus_med,inscrits,part_impose,chom_tot,etrangers
0,ZA,1ère circonscription,M. SERVA Olivier,REM,"61,74 %",NaN,NaN,6/21/74,34 - Expert comptable,2,...,0.0,0.0,0.0,0.0,1.0,11830.0,76361.0,0.3690,0.237,0.0621
1,ZA,2ème circonscription,Mme BENIN Justine,DVG,"64,26 %",NaN,NaN,3/12/75,48 - Fonctionnaire de catégorie B,2,...,0.0,0.0,0.0,0.0,0.0,11830.0,86914.0,0.3690,0.237,0.0621
2,ZA,3ème circonscription,M. MATHIASIN Max,DVG,"65,15 %",NaN,NaN,2/24/56,52 - Employé (autres entrep. publiques),2,...,0.0,0.0,0.0,0.0,0.0,11830.0,83768.0,0.3690,0.237,0.0621
3,ZA,4ème circonscription,Mme VAINQUEUR-CHRISTOPHE Hélène,SOC,"61,61 %",NaN,NaN,5/6/56,28 - Pharmacien,2,...,0.0,0.0,0.0,0.0,0.0,11830.0,69302.0,0.3690,0.237,0.0621
4,ZB,1ère circonscription,Mme MANIN Josette,DVG,"54,89 %",NaN,NaN,3/16/50,61 - Retraité salarié privé,2,...,0.0,0.0,0.0,0.0,0.0,12709.7,79411.0,0.3801,0.180,0.0621
5,ZB,2ème circonscription,M. AZEROT Bruno Nestor,DVG,"55,78 %",O,DEP,7/22/61,09 - Commercant,2,...,1.0,1.0,0.0,0.0,0.0,15331.7,81795.0,0.4326,0.180,0.0621
6,ZB,3ème circonscription,M. LETCHIMY Serge,DVG,"73,94 %",O,DEP,1/13/53,63 - Retraité fonct.publique (sf enseig.),2,...,1.0,1.0,0.0,0.0,0.0,13802.3,65881.0,0.4148,0.180,0.0621
7,ZB,4ème circonscription,M. NILOR Jean-Philippe,REG,"68,02 %",O,DEP,5/15/65,57 - Sans profession déclarée,2,...,1.0,1.0,0.0,0.0,0.0,13801.7,83725.0,0.3982,0.180,0.0621
8,ZC,1ère circonscription,M. SERVILLE Gabriel,DVG,"51,33 %",O,DEP,9/27/59,44 - Profession rattachée à l'enseignt.,2,...,1.0,1.0,0.0,0.0,0.0,11830.0,53535.0,0.3690,0.230,0.0621
9,ZC,2ème circonscription,M. ADAM Lénaïck,REM,"50,21 %",NaN,NaN,2/19/92,50 - Cadre sup. (entreprises publiques),2,...,0.0,0.0,0.0,0.0,1.0,11830.0,38500.0,0.3690,0.230,0.0621


In [166]:
for div in json_f["Nuance"].unique():
    sub_json = json_f[json_f["Nuance"]==div]
    sub_json.to_json("json_viz/json_bloc_"+div+".json", orient="index", force_ascii=False)

In [167]:
json_f = json_f.rename(columns={"Nuance":"Id"})

In [168]:
json_f["name"] = np.nan

json_f.ix[json_f["Id"]=="COM", "name"] = "Parti Communiste Français"
json_f.ix[json_f["Id"]=="FI", "name"] =  "La France Insoumise"
json_f.ix[json_f["Id"]=="SOC", "name"] =  "Parti Socialiste"
json_f.ix[json_f["Id"]=="RDG", "name"] =  "Les Radicaux de Gauche"
json_f.ix[json_f["Id"]=="DVG", "name"] =  "Divers Gauche"
json_f.ix[json_f["Id"]=="ECO", "name"] =  "Ecologistes"
json_f.ix[json_f["Id"]=="DIV", "name"] =  "Divers"
json_f.ix[json_f["Id"]=="REG", "name"] =  "Divers"
json_f.ix[json_f["Id"]=="REM", "name"] =  "La République en Marche"
json_f.ix[json_f["Id"]=="MDM", "name"] =  "Mouvement Démocrate"
json_f.ix[json_f["Id"]=="LR", "name"] =  "Les Républicains"
json_f.ix[json_f["Id"]=="DVD", "name"] =  "Divers Droite"
json_f.ix[json_f["Id"]=="FN", "name"] = "Front National"
json_f.ix[json_f["Id"]== "EXD", "name"] =   "EXD"

In [201]:
#french = json_f.groupby(['Id',"name"]).apply(lambda x: x.to_json(orient='records', 
#                                                                 force_ascii=False, lines=True))
french = json_f.groupby(['Id',"name"]).apply(lambda x: x.to_dict(orient='records'))
french = french.rename("seats")
french = french.reset_index()

In [230]:
# création des statistiques de parti

extended_df = pd.get_dummies(json_f, columns=['Profession', 'sexe'])

extended_df['categorie_age'] = pd.cut(extended_df['age'], bins=[0, 30, 40, 50, 60, 70, 100],

                                      labels=['30 et -', '31-40', '41-50', '51-60', '61-70', '71 et +'])

extended_df = pd.get_dummies(extended_df, columns=['categorie_age'])

#print(extended_df.columns)

df_partis = pd.DataFrame(extended_df.groupby('Id')['sexe_F',

       'sexe_M', 'categorie_age_30 et -', 'categorie_age_31-40',

       'categorie_age_41-50', 'categorie_age_51-60', 'categorie_age_61-70',

       'categorie_age_71 et +'].sum())

df_partis = pd.concat([df_partis, pd.DataFrame(extended_df['Id'].value_counts())

                       .rename(columns={'Id': 'total_deputes'})], axis=1)

# sauvegarde en format json
#df_partis.to_json('stats_partis.json', orient='index')
# manque les professions

In [202]:
#type(french.ix[0, "seats"])

list

In [231]:
df_partis = df_partis.reset_index()
df_partis = df_partis.rename(columns={"index":"Id"})

In [233]:
french.shape

(14, 3)

In [237]:
french_2 = pd.merge(df_partis,french, on='Id')

In [239]:
french_2 = french_2.set_index("Id")
french_2 = french_2.reindex(["COM","FI","SOC","RDG","DVG","ECO","DIV","REG",
                            "REM","MDM","UDI","LR","DVD","FN","EXD"])

In [241]:
french_2 = french_2.reset_index()
french_2

,Id,sexe_F,sexe_M,categorie_age_30 et -,categorie_age_31-40,categorie_age_41-50,categorie_age_51-60,categorie_age_61-70,categorie_age_71 et +,total_deputes,name,seats
0,COM,2.0,8.0,0.0,1.0,3.0,3.0,3.0,0.0,10.0,Parti Communiste Français,"[{'age': 59, 'Nom_de_la_Circonscription': '1èr..."
1,FI,7.0,10.0,3.0,4.0,6.0,3.0,1.0,0.0,17.0,La France Insoumise,"[{'age': 41, 'Nom_de_la_Circonscription': '1èr..."
2,SOC,12.0,18.0,0.0,1.0,10.0,12.0,7.0,0.0,30.0,Parti Socialiste,"[{'age': 61, 'Nom_de_la_Circonscription': '4èm..."
3,RDG,3.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,3.0,Les Radicaux de Gauche,"[{'age': 52, 'Nom_de_la_Circonscription': 'Sai..."
4,DVG,3.0,9.0,0.0,0.0,3.0,5.0,4.0,0.0,12.0,Divers Gauche,"[{'age': 42, 'Nom_de_la_Circonscription': '2èm..."
5,ECO,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,Ecologistes,"[{'age': 59, 'Nom_de_la_Circonscription': '2èm..."
6,DIV,1.0,2.0,0.0,2.0,0.0,1.0,0.0,0.0,3.0,Divers,"[{'age': 34, 'Nom_de_la_Circonscription': '9èm..."
7,REG,0.0,5.0,0.0,0.0,3.0,1.0,0.0,1.0,5.0,Divers,"[{'age': 52, 'Nom_de_la_Circonscription': '4èm..."
8,REM,144.0,164.0,27.0,80.0,102.0,74.0,23.0,2.0,308.0,La République en Marche,"[{'age': 43, 'Nom_de_la_Circonscription': '1èr..."
9,MDM,19.0,23.0,1.0,3.0,14.0,11.0,13.0,0.0,42.0,Mouvement Démocrate,"[{'age': 40, 'Nom_de_la_Circonscription': '7èm..."


In [242]:
french_2.to_json("json_viz/french.json", orient="records",  force_ascii=False)